In [7]:
import pandas as pd
import numpy as np
import yfinance as yf
pd.options.mode.copy_on_write = True
pd.options.display.float_format = '{:.2f}'.format



class PortfolioBacktester:
    """
    A class for backtesting a portfolio with quarterly changing weights and calculating risk metrics.
    """
    
    def __init__(self, start_date, end_date, stock_symbols, risk_free_rate=0, initial_investment=10000, reset_model=False, ):
        """
        Initializes the PortfolioBacktester class.

        Args:
            prices: A pandas DataFrame with dates (index) and stock prices as columns.
            weights: A pandas DataFrame with dates (index) and weights for each stock as columns.
            risk_free_rate: Risk-free rate of return (default: 0).
        """
        self.reset_model = reset_model
        self.initial_investment = initial_investment
        self.main_result = pd.DataFrame()
        stock_data = yf.download(stock_symbols, start=start_date, end=end_date)
        stock_data = stock_data[:]["Adj Close"]

        self.prices = stock_data
        self.risk_free_rate = risk_free_rate

    def load_weights(self, weights):
        self.weights = weights

    def _validate_data(self):
        """
        Performs data validation (date format, quarterly weights).
        """
        # self.prices.index = pd.to_datetime(self.prices.index)
        # self.weights.index = pd.to_datetime(self.weights.index)

        # if len(self.weights.resample("Q").size()) != len(self.weights):
        #     raise ValueError("Weights must change quarterly")

    def backtest(self, start_date, end_date):
        """
        Backtests the portfolio for the specified period and calculates performance metricsel.s.

        Args:
            start_date: Start date for backtesting (string in YYYY-MM-DD format).
            end_date: End date for backtesting (string in YYYY-MM-DD format).

        Returns:
            A pandas DataFrame with portfolio performance metrics.
        """
        # Filter data for backtesting period
        prices_filtered = self.prices.loc[start_date:end_date]
        weights_filtered = self.weights.loc[start_date:end_date]

        # Daily portfolio value
        portfolio_value = (prices_filtered * weights_filtered).sum(axis=1)
        # portfolio_value.dropna(how="any", inplace=True)
        # print(portfolio_value)
        # Daily returns
        daily_returns = portfolio_value.pct_change()
        daily_returns[0] = 0
        
        investment_Value = (daily_returns.cumsum()+ 1)*10000
        

        # Expected Return (annualized)
        expected_return = daily_returns.mean() * 252


        # Volatility (annualized)
        volatility = daily_returns.std() * np.sqrt(252)

        # Max Drawdown
        max_drawdown = (portfolio_value / portfolio_value.cummax()).min() -1
        # max_drawdown = 1 - (portfolio_value / portfolio_value.cummin()).min()

        # Sharpe Ratio
        sharpe_ratio = (expected_return - self.risk_free_rate) / volatility

        # Calculate CAGR
        start_value = portfolio_value.iloc[0]
        end_value = portfolio_value.iloc[-1]
        num_years = len(portfolio_value) / 252  # Assuming 252 trading days in a year
        cagr = (end_value / start_value) ** (1 / num_years) - 1

        # Print CAGR

        # Sortino Ratio (assuming negative returns for downside deviation)
        downside_returns = daily_returns[daily_returns < 0]
        sortino_ratio = (
            (expected_return - self.risk_free_rate)
            / downside_returns.std()
            * np.sqrt(252)
        )

        # Omega Ratio (assuming positive returns for upside potential)
        upside_returns = daily_returns[daily_returns > 0]
        omega_ratio = (
            upside_returns.mean() / abs(downside_returns.mean()) * np.sqrt(252)
        )

        # Combine results
        results = pd.DataFrame(
            {
                "Portfolio Unit Value": portfolio_value,
                "Daily Return": daily_returns,
                "Investment Value": investment_Value,
                "Expected Return": expected_return,
                "Volatility": volatility,
                "Max Drawdown": max_drawdown,
                "Sharpe Ratio": sharpe_ratio,
                "Sortino Ratio": sortino_ratio,
                "Omega Ratio": omega_ratio,
                "CAGR": cagr
            }
        )
        if self.reset_model:
            self.main_result = results
        else:
            self.main_result = pd.concat([self.main_result, results])
            
    def final_output(self):
        return self.main_result

In [8]:
start_date = pd.to_datetime("2018-01-01")
end_date = pd.to_datetime("2023-12-31")

stock_symbols = ["AAPL","GOOG","MSFT"]

In [9]:
class PortfolioOptimizer(PortfolioBacktester):
    def __init__(self, start_date, end_date, stock_symbols, risk_free_rate=0, reset_model=False):
        
        super().__init__(start_date, end_date, stock_symbols, risk_free_rate, reset_model, )
        self.stocks_prices = self.prices

        
    def equal_weighted(self):   
        weights = pd.DataFrame(1 / len(stock_symbols), index=self.stocks_prices.index, columns=stock_symbols)
        self.load_weights(weights)
        self.backtest(start_date, end_date)
        return self.final_output()
    
    # Maximum Diversification Portfolio Optimization
    def max_diversification(self):

        import riskfolio as rp

        def q_rebal(daily_returns):
            

# Building the portfolio object
            port = rp.Portfolio(returns=daily_returns)

            port.assets_stats(method_mu='hist', method_cov='hist', d=0.94)
            # Estimate optimal portfolio:
            port.mu = pd.DataFrame(
                np.sqrt(np.diag(port.cov)).reshape(-1, len(daily_returns.columns)), columns=daily_returns.columns)

            w1 = port.optimization(model='Classic', rm='MV',
                                obj='Sharpe', rf=0, l=0, hist=True)
            # Estimate points in the efficient frontier mean - semi standard deviation

            # Estimate the risk parity portfolio for semi standard deviation
            weights = w1.T

            

            return weights
                
        daily_returns = self.prices.pct_change(fill_method=None)

        Weights = pd.DataFrame()

        weights_df = daily_returns.resample("Q")
        
        for name, group in weights_df:
            i = daily_returns[:group.index[-1]]
            i.fillna(0, inplace=True)
            w = q_rebal(i)
            # w = pd.DataFrame.from_dict(w, orient='index', columns=['Weight'],).T
            w.index = pd.to_datetime([name])
            # print(i.index[-1])
            Weights = pd.concat([Weights, w])
            Weights = Weights.drop_duplicates().reindex(self.prices.index)

            Weights = Weights.ffill()
            
        self.load_weights(Weights)
        self.backtest(start_date, end_date)
        return self.final_output()
        raise NotImplementedError()

    # Risk Parity Portfolio Optimisation
    def min_risk(self):

        import riskfolio as rp
        def q_rebal(daily_returns):

            port = rp.Portfolio(returns=daily_returns)

            port.assets_stats(method_mu='hist', method_cov='hist', d=0.94)
            w = port.optimization(model='Classic',obj='MinRisk', rm='MV', rf=0, hist=True)

            weights = w
            return weights
                
        daily_returns = self.prices.pct_change(fill_method=None)

        Weights = pd.DataFrame()

        weights_df = daily_returns.resample("Q")
        
        for name, group in weights_df:
            i = daily_returns[:group.index[-1]]
            i.fillna(0, inplace=True)
            w = q_rebal(i)
            w = pd.DataFrame(w).T
            w.index = pd.to_datetime([name])
            # print(i.index[-1])
            Weights = pd.concat([Weights, w])
            Weights = Weights.drop_duplicates().reindex(self.prices.index)

            Weights = Weights.ffill()
            
        self.load_weights(Weights)
        self.backtest(start_date, end_date)
        return self.final_output()


    # Mean Variance Portfolio Optimisation
    def mean_variance(self):

        import riskfolio as rp
        
        def q_rebal(daily_returns):
            port = rp.Portfolio(returns=daily_returns)
            port.assets_stats(method_mu='hist', method_cov='hist', d=0.94)
            w1 = port.optimization(model='Classic', rm='MV',
                       obj='Sharpe', rf=0, l=0, hist=True)
            return w1
                
        daily_returns = self.prices.pct_change(fill_method=None)

        Weights = pd.DataFrame()

        weights_df = daily_returns.resample("Q")
        
        for name, group in weights_df:
            i = daily_returns[:group.index[-1]]
            i.fillna(0, inplace=True)
            w = q_rebal(i)
            w = pd.DataFrame(w).T
            w.index = pd.to_datetime([name])
            # print(i.index[-1])
            Weights = pd.concat([Weights, w])
            Weights = Weights.drop_duplicates().reindex(self.prices.index)

            Weights = Weights.ffill()
            
        self.load_weights(Weights)
        self.backtest(start_date, end_date)
        return self.final_output()
        raise NotImplementedError()

    # Mean-Absolute Deviation Portfolio Optimisation
    def MAD(self):

        import riskfolio as rp

        def q_rebal(daily_returns):

            port = rp.Portfolio(returns=daily_returns)

            port.assets_stats(method_mu='hist', method_cov='hist', d=0.94)
            w = port.optimization(model='Classic', rm='MAD', rf=0, hist=True)

            weights = w
            return weights
                
        daily_returns = self.prices.pct_change(fill_method=None)

        Weights = pd.DataFrame()

        weights_df = daily_returns.resample("Q")
        
        for name, group in weights_df:
            i = daily_returns[:group.index[-1]]
            i.fillna(0, inplace=True)
            w = q_rebal(i)
            # w = pd.DataFrame.from_dict(w, orient='index', columns=['Weight'],).T
            w = pd.DataFrame(w).T
            
            print(w)
            w.index = pd.to_datetime([name])
            # print(i.index[-1])
            Weights = pd.concat([Weights, w])
            Weights = Weights.drop_duplicates().reindex(self.prices.index)

            Weights = Weights.ffill()
            
        self.load_weights(Weights)
        self.backtest(start_date, end_date)
        return self.final_output()


    # Minimax Portfolio Optimisation
    def minimax(self):

        import riskfolio as rp

        def q_rebal(daily_returns):

            port = rp.Portfolio(returns=daily_returns)

            port.assets_stats(method_mu='hist', method_cov='hist', d=0.94)
            w = port.optimization(model='Classic', rm='WR', rf=0, hist=True)

            weights = w
            return weights
                
        daily_returns = self.prices.pct_change(fill_method=None)

        Weights = pd.DataFrame()

        weights_df = daily_returns.resample("Q")
        
        for name, group in weights_df:
            i = daily_returns[:group.index[-1]]
            i.fillna(0, inplace=True)
            w = q_rebal(i)
            # w = pd.DataFrame.from_dict(w, orient='index', columns=['Weight'],).T
            w = pd.DataFrame(w).T
            
            print(w)
            w.index = pd.to_datetime([name])
            # print(i.index[-1])
            Weights = pd.concat([Weights, w])
            Weights = Weights.drop_duplicates().reindex(self.prices.index)

            Weights = Weights.ffill()
            
        self.load_weights(Weights)
        self.backtest(start_date, end_date)
        return self.final_output()
        raise NotImplementedError()

    # Lower partial moment Portfolio Optimisation
    def lower_partial_moment(self):

        import riskfolio as rp

        def q_rebal(daily_returns):

            port = rp.Portfolio(returns=daily_returns)

            port.assets_stats(method_mu='hist', method_cov='hist', d=0.94)
            w = port.optimization(model='Classic', rm='SLPM', rf=0, hist=True)

            weights = w
            return weights
                
        daily_returns = self.prices.pct_change(fill_method=None)

        Weights = pd.DataFrame()

        weights_df = daily_returns.resample("Q")
        
        for name, group in weights_df:
            i = daily_returns[:group.index[-1]]
            i.fillna(0, inplace=True)
            w = q_rebal(i)
            # w = pd.DataFrame.from_dict(w, orient='index', columns=['Weight'],).T
            w = pd.DataFrame(w).T
            
            print(w)
            w.index = pd.to_datetime([name])
            # print(i.index[-1])
            Weights = pd.concat([Weights, w])
            Weights = Weights.drop_duplicates().reindex(self.prices.index)

            Weights = Weights.ffill()
            
        self.load_weights(Weights)
        self.backtest(start_date, end_date)
        return self.final_output()
        raise NotImplementedError()

In [10]:
# p = PortfolioOptimizer(stock_symbols=stock_symbols, reset_model=True)
# res1 = p.equal_weighted()
# res = p.mean_variance()
# res3 = p.max_diversification()
# res = p.minimax()
# res = p.risk_parity()
# res = p.lower_partial_moment()


# p = PortfolioOptimizer(stock_symbols=stock_symbols, reset_model=True)
# display(res)

In [11]:
# plt = res["Investment Value"]
# plt.plot()




In [12]:
# p.weights

In [13]:
class PortfolioOptimizer_ML(PortfolioOptimizer):
    def __init__(self, start_date, end_date, stock_symbols, risk_free_rate=0, reset_model=False,):
        
        super().__init__(start_date, end_date, stock_symbols, risk_free_rate, reset_model, )
        self.stocks_prices = self.prices
        
    def RF(self):
        import numpy as np
        from sklearn.ensemble import RandomForestRegressor

        def portfolio_optimization(returns, target_return=self.risk_free_rate):
            # Convert returns to a numpy array
            returns = np.array(returns)
            
            # Compute mean returns and covariance matrix
            mean_returns = np.mean(returns, axis=0)
            cov_matrix = np.cov(returns.T)
            
            # Create a dataset of random portfolios
            num_portfolios = 10000
            portfolios = np.random.uniform(0, 1, (num_portfolios, len(mean_returns)))
            portfolios = portfolios / np.sum(portfolios, axis=1)[:, np.newaxis]
            
            # Compute returns and volatilities for each portfolio
            portfolio_returns = np.dot(portfolios, mean_returns)
            portfolio_volatilities = []
            for portfolio in portfolios:
                portfolio_volatility = np.sqrt(np.dot(portfolio, np.dot(cov_matrix, portfolio)))
                portfolio_volatilities.append(portfolio_volatility)
            portfolio_volatilities = np.array(portfolio_volatilities)
            
            # Compute Sharpe ratios
            sharpe_ratios = (portfolio_returns - target_return) / portfolio_volatilities
            
            # Train the Random Forest model
            rf = RandomForestRegressor(n_estimators=100, random_state=42)
            rf.fit(portfolios, sharpe_ratios)
            
            # Optimize the portfolio weights
            max_sharpe_weight = rf.predict(np.eye(len(mean_returns)))
            max_sharpe_weight = max_sharpe_weight / np.sum(max_sharpe_weight)
            
            # Ensure weights are between 0 and 1
            max_sharpe_weight = np.clip(max_sharpe_weight, 0, 1)
            
            # Normalize weights to sum up to 1
            max_sharpe_weight = max_sharpe_weight / np.sum(max_sharpe_weight)
            
            return max_sharpe_weight
                
        daily_returns = self.prices.pct_change(fill_method=None)

        Weights = pd.DataFrame()

        weights_df = daily_returns.resample("Q")
        
        for name, group in weights_df:
            i = daily_returns[:group.index[-1]]
            i.fillna(0, inplace=True)
            w = portfolio_optimization(i)
            # w = pd.DataFrame.from_dict(w, orient='index', columns=['Weight'],).T
            w = pd.DataFrame(w, index=daily_returns.columns).T
            
            w.index = pd.to_datetime([name])
            # print(i.index[-1])
            # display(w)
            
            Weights = pd.concat([Weights, w])
            Weights = Weights.drop_duplicates().reindex(self.prices.index)

            Weights = Weights.ffill()
            
        self.load_weights(Weights)
        self.backtest(start_date, end_date)
        return self.final_output()
        raise NotImplementedError()
    
    def SVM(self):
        import numpy as np
        from sklearn.svm import SVR

        def portfolio_optimization(returns, target_return = self.risk_free_rate):
            # Convert returns to a numpy array
            returns = np.array(returns)
            
            # Compute mean returns and covariance matrix
            mean_returns = np.mean(returns, axis=0)
            cov_matrix = np.cov(returns.T)
            
            # Create a dataset of random portfolios
            num_portfolios = 10000
            portfolios = np.random.uniform(0, 1, (num_portfolios, len(mean_returns)))
            portfolios = portfolios / np.sum(portfolios, axis=1)[:, np.newaxis]
            
            # Compute returns and volatilities for each portfolio
            portfolio_returns = np.dot(portfolios, mean_returns)
            portfolio_volatilities = []
            for portfolio in portfolios:
                portfolio_volatility = np.sqrt(np.dot(portfolio, np.dot(cov_matrix, portfolio)))
                portfolio_volatilities.append(portfolio_volatility)
            portfolio_volatilities = np.array(portfolio_volatilities)
            
            # Compute Sharpe ratios
            sharpe_ratios = (portfolio_returns - target_return) / portfolio_volatilities
            
            # Train the SVR model
            svr = SVR(kernel='rbf', C=1000, epsilon=0.01)
            svr.fit(portfolios, sharpe_ratios)
            
            # Optimize the portfolio weights
            max_sharpe_weight = svr.predict(np.eye(len(mean_returns)))
            max_sharpe_weight = max_sharpe_weight / np.sum(max_sharpe_weight)
            
            # Ensure weights are between 0 and 1
            max_sharpe_weight = np.clip(max_sharpe_weight, 0, 1)
            
            # Normalize weights to sum up to 1
            max_sharpe_weight = max_sharpe_weight / np.sum(max_sharpe_weight)
            
            return max_sharpe_weight
                
        daily_returns = self.prices.pct_change(fill_method=None)

        Weights = pd.DataFrame()

        weights_df = daily_returns.resample("Q")
        
        for name, group in weights_df:
            i = daily_returns[:group.index[-1]]
            i.fillna(0, inplace=True)
            w = portfolio_optimization(i)
            # w = pd.DataFrame.from_dict(w, orient='index', columns=['Weight'],).T
            w = pd.DataFrame(w, index=daily_returns.columns).T
            
            w.index = pd.to_datetime([name])
            # print(i.index[-1])
            display(w)
            
            Weights = pd.concat([Weights, w])
            Weights = Weights.drop_duplicates().reindex(self.prices.index)

            Weights = Weights.ffill()
            
        self.load_weights(Weights)
        self.backtest(start_date, end_date)
        return self.final_output()
        raise NotImplementedError()
    
    def MVF(self):
        import numpy as np
        import cvxopt as cv
        def portfolio_optimization(returns, target_return=self.risk_free_rate, forecast_returns=None):
    # Convert returns to a numpy array
            returns = np.array(returns)

            # Compute mean returns and covariance matrix
            mean_returns = np.mean(returns, axis=0)
            cov_matrix = np.cov(returns.T)

            # If forecast returns are provided, use them instead of historical returns
            if forecast_returns is not None:
                mean_returns = forecast_returns

            # Solve the optimization problem using quadratic programming
            num_assets = len(mean_returns)
            P = cv.matrix(cov_matrix.tolist())
            q = cv.matrix(np.zeros((num_assets, 1)))
            G = cv.matrix(np.vstack((np.eye(num_assets), -np.eye(num_assets))).tolist())
            h = cv.matrix(np.ones((2 * num_assets, 1)))
            A = cv.matrix(np.ones((1, num_assets)))
            b = cv.matrix(1.0)

            # Solve the optimization problem
            sol = cv.solvers.qp(P, q, G, h, A, b)
            if sol['status'] != 'optimal':
                raise ValueError('Optimization problem could not be solved')

            # Get the optimal portfolio weights
            optimal_weights = np.array(sol['x']).flatten()

            # Ensure weights are between 0 and 1
            optimal_weights = np.clip(optimal_weights, 0, 1)

            # Normalize weights to sum up to 1
            optimal_weights = optimal_weights / np.sum(optimal_weights)

            return optimal_weights
                
        daily_returns = self.prices.pct_change(fill_method=None)

        Weights = pd.DataFrame()

        weights_df = daily_returns.resample("Q")
        
        for name, group in weights_df:
            i = daily_returns[:group.index[-1]]
            i.fillna(0, inplace=True)
            w = portfolio_optimization(i)
            # w = pd.DataFrame.from_dict(w, orient='index', columns=['Weight'],).T
            w = pd.DataFrame(w, index=daily_returns.columns).T
            
            w.index = pd.to_datetime([name])
            # print(i.index[-1])
            display(w)
            
            Weights = pd.concat([Weights, w])
            Weights = Weights.drop_duplicates().reindex(self.prices.index)

            Weights = Weights.ffill()
            
        self.load_weights(Weights)
        self.backtest(start_date, end_date)
        return self.final_output()
        raise NotImplementedError()



In [14]:
# p = PortfolioOptimizer_ML(stock_symbols, reset_model=True)
# p.SVM()

In [17]:
class PortfolioOptimizer_DL(PortfolioOptimizer_ML):
    def __init__(self, start_date, end_date, stock_symbols, risk_free_rate=0, reset_model=False, ):
    
        super().__init__(start_date, end_date, stock_symbols, risk_free_rate, reset_model)
        self.stocks_prices = self.prices
        
    def DMLP(self):
        import numpy as np
        import pandas as pd
        import tensorflow as tf
        from tensorflow.keras.models import Sequential
        from tensorflow.keras.layers import Dense
        from tensorflow.keras.optimizers import Adam
        from tensorflow.keras.callbacks import EarlyStopping
        
        
        
        def portfolio_optimization(returns, target_return=self.risk_free_rate):
            # Convert returns to a numpy array
            returns = np.array(returns)

            # Compute mean returns and covariance matrix
            mean_returns = np.mean(returns, axis=0)
            cov_matrix = np.cov(returns.T)

            # Create a dataset of random portfolios
            num_portfolios = 10000
            portfolios = np.random.uniform(0, 1, (num_portfolios, len(mean_returns)))
            portfolios = portfolios / np.sum(portfolios, axis=1)[:, np.newaxis]

            # Compute returns and volatilities for each portfolio
            portfolio_returns = np.dot(portfolios, mean_returns)
            portfolio_volatilities = []
            for portfolio in portfolios:
                portfolio_volatility = np.sqrt(np.dot(portfolio, np.dot(cov_matrix, portfolio)))
                portfolio_volatilities.append(portfolio_volatility)
            portfolio_volatilities = np.array(portfolio_volatilities)

            # Compute Sharpe ratios
            sharpe_ratios = (portfolio_returns - target_return) / portfolio_volatilities

            # Train the MLP model
            mlp_model = Sequential()
            mlp_model.add(Dense(64, input_dim=len(mean_returns), activation='relu'))
            mlp_model.add(Dense(32, activation='relu'))
            mlp_model.add(Dense(1, activation='linear'))
            mlp_model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
            # mlp_model.fit(portfolios, sharpe_ratios, epochs=100, batch_size=32, verbose=0)
            
                        # Define early stopping callback
            early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

            # Fit the MLP model with early stopping
            mlp_model.fit(portfolios, sharpe_ratios, epochs=100, batch_size=32, verbose=0, callbacks=[early_stopping])

            # Optimize the portfolio weights
            max_sharpe_weight = mlp_model.predict(np.eye(len(mean_returns)))
            max_sharpe_weight = max_sharpe_weight.flatten()

            # Ensure weights are between 0 and 1
            max_sharpe_weight = np.clip(max_sharpe_weight, 0, 1)

            # Normalize weights to sum up to 1
            max_sharpe_weight = max_sharpe_weight / np.sum(max_sharpe_weight)

            return max_sharpe_weight

        daily_returns = self.prices.pct_change(fill_method=None)

        Weights = pd.DataFrame()

        weights_df = daily_returns.resample("Q")

        for name, group in weights_df:
            i = daily_returns[:group.index[-1]]
            i.fillna(0, inplace=True)
            w = portfolio_optimization(i)
            w = pd.DataFrame(w, index=daily_returns.columns).T
            w.index = pd.to_datetime([name])
            display(w)
            Weights = pd.concat([Weights, w])

        Weights = Weights.drop_duplicates().reindex(self.prices.index)
        Weights = Weights.ffill()

        self.load_weights(Weights)
        self.backtest(start_date, end_date)
        return self.final_output()

In [18]:
p = PortfolioOptimizer_DL(stock_symbols=stock_symbols, reset_model=True, start_date=start_date, end_date=end_date)
p.DMLP()

[*********************100%%**********************]  3 of 3 completed
c:\Users\1000305366\Personal\.venv\lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\1000305366\Personal\.venv\lib\site-packages\keras\src\callbacks\early_stopping.py:156: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  current = self.get_monitor_value(logs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step


Ticker,AAPL,GOOG,MSFT
2018-03-31,0.00,0.00,1.00


c:\Users\1000305366\Personal\.venv\lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\1000305366\Personal\.venv\lib\site-packages\keras\src\callbacks\early_stopping.py:156: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  current = self.get_monitor_value(logs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


Ticker,AAPL,GOOG,MSFT
2018-06-30,0.32,0.19,0.49


c:\Users\1000305366\Personal\.venv\lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\1000305366\Personal\.venv\lib\site-packages\keras\src\callbacks\early_stopping.py:156: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  current = self.get_monitor_value(logs)


KeyboardInterrupt: 